In [6]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv

In [8]:
load_dotenv() # Take enviroment variables from .env
KEY=os.getenv("OPENAI_API_KEY")

In [9]:
llm=ChatOpenAI(openai_api_key=KEY, model_name='gpt-3.5-turbo', temperature=0.5)